In [ ]:
# default_exp pyspark_utils
import os
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# Pyspark Utils

Isolate pyspark related code to make pyspark module optional.

## Imports

In [ ]:
# export
from typing import Dict, Tuple, List
import os

from loguru import logger

from pyspark import RDD, SparkContext
from pyspark.storagelevel import StorageLevel


In [ ]:
# export 
class Hdfs(object):
    """Convenience wrapper of hdfs functions for pyspark"""

    def __init__(self, sc=None):
        """ Init Hdfs instance.
        Args:
          sc: SparkContext instance.
        """
        if sc is None:
            sc = SparkContext.getOrCreate()
        self.sc = sc
        self._hadoop_conf = sc._jsc.hadoopConfiguration()

    def list(self, path):
        """Return files under path"""
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        files = fs.listStatus(p)
        return [f.getPath().toString() for f in files]

    def exists(self, path):
        """Return path is exists or not"""
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        return fs.exists(p)

    def remove(self, path):
        """Remove/delete path"""
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        return fs.delete(p, True)

    def copy(self, src_path, dst_path):
        """copy src_path to dst_path"""
        sp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(src_path)
        dp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(dst_path)
        src_fs = sp.getFileSystem(self._hadoop_conf)
        dst_fs = dp.getFileSystem(self._hadoop_conf)
        return self.sc._gateway.jvm.org.apache.hadoop.fs.FileUtil.copy(
            src_fs, sp, dst_fs, dst_path, False, True, self._hadoop_conf)

    def copyToLocalFile(self, src_path, dst_path, checksum=True):
        """copy src_path to dst_path
        added by alonso
        """
        sp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(src_path)
        dp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(dst_path)
        fs = sp.getFileSystem(self._hadoop_conf)
        fs.setVerifyChecksum(checksum)
        fs.copyToLocalFile(sp, dp)

    def copyFromLocalFile(self, src_path, dst_path, checksum=True):
        """added by alonso"""
        sp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(src_path)
        dp = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(dst_path)
        fs = dp.getFileSystem(self._hadoop_conf)
        fs.setVerifyChecksum(checksum)
        fs.copyFromLocalFile(sp, dp)

    def mkdir(self, path):
        """Make directory"""
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        return fs.mkdirs(p)

    def get_hdfs_rdd(self, path):
        return self.sc.textFile(path)

    def create(self, path):
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        return fs.create(p)

    def write(self, content, path, data_type="str"):
        p = self.sc._gateway.jvm.org.apache.hadoop.fs.Path(path)
        fs = p.getFileSystem(self._hadoop_conf)
        hdfs_writer = fs.create(p)
        if data_type == "str":
            hdfs_writer.write(bytearray(content, "utf-8"))
        else:
            hdfs_writer.write(content)
        hdfs_writer.close()


## Decorator

In [ ]:
# export

def join_dict_of_rdd(rdd_dict: Dict[str, RDD]) -> RDD:
    """Join dictionary of RDD, but not a traditional join

    1. Stack all RDD together and make as paired RDD
    2. reducebyKey, value is list of dict
    3. fill all field accorddingly, and return a RDD of dict

    Args:
        rdd_dict (Dict[str, RDD]): RDD dict

    Returns:
        RDD: Joined RDD
    """

    # create loss multiplier for inputs
    rdd_list = []
    loss_multiplier_list = []

    def _add_loss_multiplier(inp: dict, problem: str) -> dict:
        lm_name = '{}_loss_multiplier'.format(problem)
        inp[lm_name] = 1
        return inp
    for p, rdd in rdd_dict.items():
        loss_multiplier_list.append('{}_loss_multiplier'.format(p))
        rdd_list.append(
            rdd.map(lambda x, p=p: _add_loss_multiplier(x, problem=p))
        )

    # union rdds
    sc: SparkContext = SparkContext.getOrCreate()
    all_problem_rdd = sc.union(rdd_list)

    # make pair rdd
    def _make_pair_rdd(inp_dict: dict) -> Tuple[str, dict]:
        if 'record_id' not in inp_dict:
            raise KeyError(
                "Chaining problems with & without "
                "providing 'record_id' in inputs. Received keys: {}".format(inp_dict.keys()))
        return (inp_dict['record_id'], inp_dict)
    all_problem_rdd = all_problem_rdd.map(_make_pair_rdd)

    # reduce by key, fill out dict correspondingly
    loss_multiplier_list_b = sc.broadcast(loss_multiplier_list)

    def _merge_dicts(left_dict: dict, right_dict: dict):
        left_dict.update(right_dict)
        return left_dict

    def _add_dummpy_loss_multiplier(inp: dict) -> dict:
        # set loss multiplier to inform which problem
        # is available in this record
        lml = loss_multiplier_list_b.value
        for lm in lml:
            if lm not in inp:
                inp[lm] = 0
        return inp
    # MEMORY_AND_DISK
    all_problem_rdd = all_problem_rdd.persist(
        storageLevel=StorageLevel(True, True, False, False))
    all_problem_rdd = all_problem_rdd.reduceByKey(_merge_dicts).map(
        lambda x: x[1]).map(_add_dummpy_loss_multiplier)

    return all_problem_rdd


In [ ]:
sc : SparkContext= SparkContext.getOrCreate()
test_rdd_dict = {
    'p1': sc.parallelize([
        {'record_id': 0, 'a': 1},
        {'record_id': 1, 'a': 2}
    ]),
    'p2': sc.parallelize([
        {'record_id': 0, 'b': 1},
        {'record_id': 2, 'b': 2}
    ])
}
rdd = join_dict_of_rdd(test_rdd_dict)
assert rdd.collect() == [{'record_id': 0, 'a': 1, 'p1_loss_multiplier': 1, 'b': 1, 'p2_loss_multiplier': 1}, {'record_id': 1, 'a': 2, 'p1_loss_multiplier': 1, 'p2_loss_multiplier': 0}, {'record_id': 2, 'b': 2, 'p2_loss_multiplier': 1, 'p1_loss_multiplier': 0}]

In [ ]:
# export
def get_text_file_from_executor(remote_path: str, local_path: str) -> str:
    fake_rdd :RDD = SparkContext.getOrCreate().parallelize([1,2,3])
    def _read_text_file(x) -> str:
        if not os.path.exists(remote_path):
            raise FileNotFoundError('{} not found in executor'.format(remote_path))
        return [open(remote_path, 'r', encoding='utf8').read()]
    file_str_list : List[str]= fake_rdd.mapPartitions(_read_text_file).collect()
    if file_str_list:
        open(local_path, 'w', encoding='utf8').write(file_str_list[0])
    else:
        logger.warning('fetch {} to {} failed'.format(remote_path, local_path))
    
    

In [ ]:

# hide
from m3tl.test_base import PysparkTestBase

pyspark_test_base = PysparkTestBase()

# create file on executor
def create_file_on_executor():
    fake_rdd : RDD= SparkContext.getOrCreate().parallelize([1,2,3])
    def _make_text_file(x):
        open('text.txt', 'w').write('testtest')
        return [1]
    fake_rdd = fake_rdd.mapPartitions(_make_text_file)
    fake_rdd.count()
    
create_file_on_executor()
get_text_file_from_executor('text.txt', 'local_test.txt')
assert open('local_test.txt', 'r').read() == 'testtest'
os.remove('text.txt')
os.remove('local_test.txt')


2021-06-17 23:46:38.465 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem pyspark_fake_seq_tag, problem type: seq_tag
2021-06-17 23:46:38.465 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem pyspark_fake_multi_cls, problem type: multi_cls
2021-06-17 23:46:38.466 | INFO     | m3tl.base_params:register_multiple_problems:538 - Adding new problem pyspark_fake_cls, problem type: cls
2021-06-17 23:46:38.466 | WARNING  | m3tl.base_params:assign_problem:642 - base_dir and dir_name arguments will be deprecated in the future. Please use model_dir instead.
2021-06-17 23:46:38.467 | WARNING  | m3tl.base_params:prepare_dir:361 - bert_config not exists. will load model from huggingface checkpoint.
2021-06-17 23:46:44.141 | INFO     | m3tl.utils:set_phase:478 - Setting phase to train


In [ ]:
# export
def repar_rdd(rdd: RDD, rdd_count: int, example_per_par=100000, coalesce_only=True):
    """
    repar rdd based on number of example. if coalesce_only is False and expected
    partition is greater than current partition then nothing will happen
    """
    num_partition = rdd.getNumPartitions()
    expect_partition = max(1, int(rdd_count / example_per_par))
    
    if expect_partition < num_partition:
        rdd = rdd.coalesce(expect_partition)
    elif expect_partition > num_partition and coalesce_only is False:
        rdd = rdd.repartition(expect_partition)
        
    return rdd
    
